In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Send a GET request to the URL
url = "https://www.popvortex.com/music/charts/top-100-songs.php"
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Find all p tags with class 'title-artist'
title_artist_tags = soup.find_all('p', class_='title-artist')

# Extract titles and artists from the tags
titles = []
artists = []
for tag in title_artist_tags:
    title = tag.find('cite', class_='title').text.strip()
    artist = tag.find('em', class_='artist').text.strip()
    titles.append(title)
    artists.append(artist)

# Create a DataFrame
data = {'Title': titles, 'Artist': artists}
itunes_df = pd.DataFrame(data)

# df.to_csv('music_chart_data.csv', index=False)

# Display the DataFrame
itunes_df.head()


,Title,Artist
0,Lovin On Me,Jack Harlow
1,Lose Control,Teddy Swims
2,Beautiful Things,Benson Boone
3,Cruel Summer,Taylor Swift
4,Snooze,SZA


# Lab 2 expand data



In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Send a GET request to the URL
url = "https://www.billboard.com/charts/hot-100/"
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Find all list items with class 'o-chart-results-list__item'
song_items = soup.find_all('li', class_='o-chart-results-list__item')

# Extract titles and artists from the list items
data = {'Title': [], 'Artist': []}
for item in song_items:
    title_elem = item.find('h3', class_='c-title')
    artist_elem = item.find('span', class_='c-label')
    if title_elem and artist_elem:
        title = title_elem.text.strip()
        artist = artist_elem.text.strip()
        data['Title'].append(title)
        data['Artist'].append(artist)

# Create a DataFrame
billboard_df = pd.DataFrame(data)

# Display the DataFrame
billboard_df.tail(20)


,Title,Artist
80,Standing Next To You,Jung Kook
81,Perro Negro,Bad Bunny & Feid
82,Mmhmm,BigXthaPlug
83,Oklahoma Smokeshow,Zach Bryan
84,My Eyes,Travis Scott
85,Coal,Dylan Gossett
86,A Cold Sunday,Lil Yachty
87,All Of Me,21 Savage
88,N.H.I.E.,21 Savage & Doja Cat
89,Should've Wore A Bonnet,21 Savage & Brent Faiyaz


In [16]:
# Concatenate both DataFrames vertically
combined_df = pd.concat([itunes_df, billboard_df], ignore_index=True)

# Display the combined DataFrame
print(combined_df)


                       Title                       Artist
0             TEXAS HOLD 'EM                      Beyoncé
1               Lose Control                  Teddy Swims
2           Beautiful Things                 Benson Boone
3                    Flowers                  Miley Cyrus
4    Turn the Lights Back On                   Billy Joel
..                       ...                          ...
195                 Worth It         Offset & Don Toliver
196        Northern Attitude       Noah Kahan With Hozier
197          Scared To Start              Michael Marcagi
198               First Love  Oscar Ortiz X Edgardo Nunez
199            Where It Ends             Bailey Zimmerman

[200 rows x 2 columns]


In [17]:
# Save the merged DataFrame to a CSV file
combined_df.to_csv('combined_df.csv', index=False)

# Part 2

A list with the different kind of datasets available in data.gov.uk: url = 'https://data.gov.uk/'

In [19]:
import requests
from bs4 import BeautifulSoup

# Send a GET request to the URL
url = "https://www.data.gov.uk/"
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Find the unordered list with class 'govuk-list dgu-topics__list'
ul = soup.find('ul', class_='govuk-list dgu-topics__list')

# Extract all list items from the unordered list
dataset_topics = ul.find_all('li')

# Iterate through each list item and print the topic and description
for topic in dataset_topics:
    topic_name = topic.find('h3', class_='govuk-heading-s dgu-topics__heading').text.strip()
    description = topic.find('p', class_='govuk-body').text.strip()
    print("Topic:", topic_name)
    print("Description:", description)
    print()


Topic: Business and economy
Description: Small businesses, industry, imports, exports and trade

Topic: Crime and justice
Description: Courts, police, prison, offenders, borders and immigration

Topic: Defence
Description: Armed forces, health and safety, search and rescue

Topic: Education
Description: Students, training, qualifications and the National Curriculum

Topic: Environment
Description: Weather, flooding, rivers, air quality, geology and agriculture

Topic: Government
Description: Staff numbers and pay, local councillors and department business plans

Topic: Government spending
Description: Includes all payments by government departments over £25,000

Topic: Health
Description: Includes smoking, drugs, alcohol, medicine performance and hospitals

Topic: Mapping
Description: Addresses, boundaries, land ownership, aerial photographs, seabed and land terrain

Topic: Society
Description: Employment, benefits, household finances, poverty and population

Topic: Towns and cities
De

Display the top 10 languages by number of native speakers stored in a pandas dataframe: url = 'https://en.wikipedia.org/wiki/List_of_languages_by_number_of_native_speakers'

https://en.wikipedia.org/wiki/List_of_languages_by_number_of_native_speakers

In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Send a GET request to the URL
url = "https://en.wikipedia.org/wiki/List_of_languages_by_number_of_native_speakers"
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table with class 'wikitable sortable'
table = soup.find('table', class_='wikitable sortable')

# Extract the rows from the table
rows = table.find_all('tr')[1:11]  # Skip the header row, take only top 10 languages

# Initialize lists to store language data
languages = []
native_speakers = []

# Iterate through each row and extract language data
for row in rows:
    columns = row.find_all('td')
    language = columns[0].text.strip()
    speakers = columns[1].text.strip()
    languages.append(language)
    native_speakers.append(speakers)

# Create a pandas DataFrame
data = {'Language': languages, 'Native Speakers (in millions)': native_speakers}
df = pd.DataFrame(data)

# Display the DataFrame
print(df)


  Language Native Speakers (in millions)
0        1              Mandarin Chinese
1        2                       Spanish
2        3                       English
3        3                        Arabic
4        5                         Hindi
5        6                       Bengali
6        7                    Portuguese
7        8                       Russian
8        9                      Japanese
9       10               Western Punjabi


Retrieve an arbitrary Wikipedia page of "Python" and create a list of links on that page: url ='https://en.wikipedia.org/wiki/Python'

In [25]:
import requests
from bs4 import BeautifulSoup

# URL of the Wikipedia page on Python
url = 'https://en.wikipedia.org/wiki/Python'

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the div element with id="mw-content-text"
    content_div = soup.find('div', id='mw-content-text')
    
    # Initialize a list to store links
    links = []
    
    # Find all <a> tags within the content_div
    for link in content_div.find_all('a', href=True):
        links.append(link['href'])
    
    # Display the list of links
    print(links)
else:
    print('Failed to retrieve data from the website')


['https://en.wiktionary.org/wiki/Python', 'https://en.wiktionary.org/wiki/python', '/w/index.php?title=Python&action=edit&section=1', '/wiki/Pythonidae', '/wiki/Python_(genus)', '/wiki/Python_(mythology)', '/w/index.php?title=Python&action=edit&section=2', '/wiki/Python_(programming_language)', '/wiki/CMU_Common_Lisp', '/wiki/PERQ#PERQ_3', '/w/index.php?title=Python&action=edit&section=3', '/wiki/Python_of_Aenus', '/wiki/Python_(painter)', '/wiki/Python_of_Byzantium', '/wiki/Python_of_Catana', '/wiki/Python_Anghelo', '/w/index.php?title=Python&action=edit&section=4', '/wiki/Python_(Efteling)', '/wiki/Python_(Busch_Gardens_Tampa_Bay)', '/wiki/Python_(Coney_Island,_Cincinnati,_Ohio)', '/w/index.php?title=Python&action=edit&section=5', '/wiki/Python_(automobile_maker)', '/wiki/Python_(Ford_prototype)', '/w/index.php?title=Python&action=edit&section=6', '/wiki/Python_(missile)', '/wiki/Python_(nuclear_primary)', '/wiki/Colt_Python', '/w/index.php?title=Python&action=edit&section=7', '/wiki